## This notebook is tested and supposted to be used in kaggle notebbok.

https://www.kaggle.com/code/alexandrmaximenko/audiollm-hw

In [ ]:
import torch
import librosa
from transformers import AutoModel, AutoProcessor, WhisperFeatureExtractor, WhisperModel, AutoTokenizer, AutoModelForCausalLM
from transformers.generation import GenerationConfig
import transformers
import pandas as pd
import json
from tqdm import tqdm
import IPython
import gc
import os
from dataclasses import dataclass
import torch.nn as nn
from typing import Optional, Tuple, List

# There might me Errors like 
# "AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'"
# It's okay and won't effect notebook executio

In [ ]:
# downloading helping visualizations
! gdown https://drive.google.com/uc?id=1k8LeWGhsn1fTXbrn9JV08lKUTOJeoEwz

# Homework: Audio Language Model Training and Evaluation (10 points)

In this homework, you will implement AudioLLM training pipeline and check it correctness with overfitting experiment.

Then, you'll take pretrained checkpoint and config for AudioLLM and write code for it's evaluation.


**Good luck! 🚀**

### Data Generation Pipeline (2 points)
Implement a QA-data generation pipeline from ASR (Automatic Speech Recognition) dataset
- Start with ASR dataset containing `(audio, transcript)` pairs
- Use the transcript to prompt an instruction-tuned LLM to generate question based on the transcript
- Use the transcript & question to prompt an instruction-tuned LLM to generate answer for this question based on transcript
- Create `(audio, question, anwer)` pairs for training

**Expected output**: A data generation script that transforms ASR data into QA-dataset + some generated samples

Expected sample format:
```
{
  "audio_path": "some_audio_path.flac",
  
  "transcript": "So today we're going to talk about how to prepare for a marathon if you only have twelve weeks...",
  
  "question": "How long does the speaker say you have to prepare for the marathon?",
  
  "answer": "Twelve weeks."
}
```

#### Grading Criteria:
* **2 points:** Completed *<TODO>* code in the `QADataGeneratorHF` + meaningfull examples of questions/answers in the last cell, completed `check_response` if needed
* **1 point:**: Completed *<TODO>* code in the `QADataGeneratorHF` + many bugs in examples like incorrect format / model refusals
* **0 points:** lack of meaningful data examples / unexecutable code

---

In [ ]:
# Download librispeech dev-clean data
! wget https://openslr.elda.org/resources/12/dev-clean.tar.gz \
  && tar -xf dev-clean.tar.gz

In [ ]:
from pathlib import Path
import json

DEV_CLEAN_ROOT = Path("/kaggle/working/LibriSpeech/dev-clean")


# Parse all transcript files into a dict: utt_id -> transcription
def load_transcripts(root: Path) -> dict:
    transcripts = {}
    for trans_file in root.rglob("*.trans.txt"):
        with open(trans_file, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue
                utt_id = parts[0]              # e.g. "84-121123-0000"
                text = " ".join(parts[1:])
                transcripts[utt_id] = text
    return transcripts

transcripts = load_transcripts(DEV_CLEAN_ROOT)
print("Loaded transcripts:", len(transcripts))

# Create JSONL with one entry per audio file
output_json = Path("/kaggle/working/dev-clean.json")
librispeech_data = []
for flac_path in sorted(DEV_CLEAN_ROOT.rglob("*.flac")):
    utt_id = flac_path.stem  # filename without .flac
    text = transcripts.get(utt_id)
    record = {
        "audio_path": str(flac_path),    # full path inside Kaggle FS
        "transcription": text,
    }
    librispeech_data.append(record)
    
with output_json.open("w", encoding="utf-8") as f:
    json.dump(librispeech_data, f)
        

print("Wrote JSONL to:", output_json)

In [ ]:
# Example of data sample
index = torch.randint(len(librispeech_data), (1,))
print('Data sample transcription:\n', librispeech_data[index]['transcription'])
IPython.display.Audio(librispeech_data[index]['audio_path'])

In [ ]:
BASE_SYSTEM_MESSAGE = (
    "You are a helpful assistant. "
    "Keep your responses concise — maximum 50 words or 2–3 sentences."
)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

class QADataGeneratorHF:
    def __init__(self, model_path: str = "Qwen/Qwen2.5-1.5B-Instruct"):
        print(f"Loading tokenizer: {model_path}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

        print(f"Loading transformers model: {model_path}")
        self.device = DEVICE
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.bfloat16 if self.device == 'cuda' else torch.float32,
            trust_remote_code=True,
        ).to(self.device)
        self.model.eval()

        self.generation_kwargs = dict(
            temperature=0.8,
            top_p=0.9,
            max_new_tokens=256,
            do_sample=True,
        )

        self.system_message = BASE_SYSTEM_MESSAGE

    
    @torch.inference_mode()
    def generate_single(self, transcription: str) -> dict:
        """Generate question and answer for a given transcription."""
        # Step 1: Generate a question based on the transcript
        question_prompt_messages = [
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": f"Based on the following transcript, generate one short question that can be answered using information from the transcript. Only output the question, nothing else.\n\nTranscript: {transcription}"}
        ]
        question_prompt = self.tokenizer.apply_chat_template(
            question_prompt_messages,
            tokenize=False,
            add_generation_prompt=True
        )
        question_enc = self.tokenizer(question_prompt, return_tensors="pt", truncation=True)
        question_input_ids = question_enc["input_ids"].to(self.device)
        question_attention_mask = question_enc["attention_mask"].to(self.device)
        
        question_output = self.model.generate(
            input_ids=question_input_ids,
            attention_mask=question_attention_mask,
            **self.generation_kwargs
        )
        question = self.tokenizer.decode(
            question_output[0][question_input_ids.shape[1]:],
            skip_special_tokens=True
        ).strip()
        
        # Step 2: Generate an answer based on the transcript and question
        answer_prompt_messages = [
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": f"Based on the following transcript, answer the question concisely. Only output the answer, nothing else.\n\nTranscript: {transcription}\n\nQuestion: {question}"}
        ]
        answer_prompt = self.tokenizer.apply_chat_template(
            answer_prompt_messages,
            tokenize=False,
            add_generation_prompt=True
        )
        answer_enc = self.tokenizer(answer_prompt, return_tensors="pt", truncation=True)
        answer_input_ids = answer_enc["input_ids"].to(self.device)
        answer_attention_mask = answer_enc["attention_mask"].to(self.device)
        
        answer_output = self.model.generate(
            input_ids=answer_input_ids,
            attention_mask=answer_attention_mask,
            **self.generation_kwargs
        )
        answer = self.tokenizer.decode(
            answer_output[0][answer_input_ids.shape[1]:],
            skip_special_tokens=True
        ).strip()
        
        return {"question": question, "answer": answer}

    def generate_batch(self, transcriptions: list[str]) -> list[dict]:
        """Generate responses for a list of transcriptions with progress bar."""
        responses = []
        for transcription in tqdm(transcriptions, desc="Generating responses"):
            response = self.generate_single(transcription)
            responses.append(response)
        
        return responses

In [ ]:
generator = QADataGeneratorHF("Qwen/Qwen2.5-1.5B-Instruct")

In [ ]:
NUM_SAMPLES = 100
librispeech_data_sample = librispeech_data[:NUM_SAMPLES]
transcriptions = [sample['transcription'] for sample in librispeech_data_sample]

generated_responses = generator.generate_batch(transcriptions)

In [ ]:
# Check your instruct data samples
index = torch.randint(NUM_SAMPLES, (1,))
print('Transcription example: ', librispeech_data_sample[index]['transcription'])
print('='*100)
print('Question example: ', generated_responses[index]['question'])
print('='*100)
print('Answer example: ', generated_responses[index]['answer'])

In [ ]:
## Add simple filtering

def check_response(response: dict) -> bool:
    """Check if the generated response is valid."""
    # Check if response has required keys
    if not isinstance(response, dict):
        return False
    if 'question' not in response or 'answer' not in response:
        return False
    
    question = response.get('question', '')
    answer = response.get('answer', '')
    
    # Check for empty responses
    if not question or not answer:
        return False
    
    # Check for model refusals or errors
    rejection_patterns = [
        "I can't", "I cannot", "I'm not able", "I am not able",
        "I don't have", "I do not have",
        "It seems like", "I'm sorry", "I am sorry",
        "I can't help", "I can't provide", "I can't create",
        "As an AI", "As a language model",
    ]
    for pattern in rejection_patterns:
        if pattern.lower() in question.lower() or pattern.lower() in answer.lower():
            return False
    
    # Check minimum length
    if len(question) < 5 or len(answer) < 2:
        return False
    
    return True

generated_samples = []
rejected_samples = []

for data_sample, response in zip(librispeech_data_sample[:NUM_SAMPLES], generated_responses):
    if check_response(response):
        data_sample.update(response)
        generated_samples.append(data_sample)
    else:
        data_sample.update(response)
        rejected_samples.append(data_sample)

print('Number of samples before filtering: ', NUM_SAMPLES)
print('Number of samples after filtering: ', len(generated_samples))
with open('instruct_data.json', 'w') as file:
    json.dump(generated_samples, file)

In [ ]:
# Check rejected samples
index = torch.randint(len(rejected_samples), (1,))
print('Transcription example: ', rejected_samples[index]['transcription'])
print('Response example: ', rejected_samples[index]['response'])

In [ ]:
# Check good samples
index = torch.randint(len(generated_samples), (1,))
print('Transcription example: ', generated_samples[index]['transcription'])
print('='*100)
print('Question example: ', generated_samples[index]['question'])
print('='*100)
print('Answer example: ', generated_samples[index]['answer'])


#### Grading Criteria:
* **2 points:** Completed *<TODO>* code in the `QADataGeneratorHF` + meaningfull examples of questions/answers in the last cell, completed `check_response` if needed
* **1 point:**: Completed *<TODO>* code in the `QADataGeneratorHF` + many bugs in examples like incorrect format / model refusals
* **0 points:** lack of meaningful data examples / unexecutable code

---

### 2. AudioLLM Model & Dataset & Overfitting (4 points)

In this section you'll need to: 
* Implement AudioLLM model with architecture, similar to UltraVox.

* Implement dataset class for this model based on QA-samples you've generated earlier

* Run overfitting experiment with your model and your dataset


#### Grading Criteria:
* **+2 points:** Completed *<TODO>* code in the `AudioAdapter` and in the `AudioLLM`, `AudioEmbeddingInsertionTests` passed
* **+1 point:** Completed *<TODO>* code in the `AudioInstructDataset` + demonstration of dataset samples
* **+1 point:** Reaching `loss < 0.2` on your overfit experiment

---

Let start with model.

**Model components**:
  - Audio encoder - Whisper
  - Audio Adapter - Linear with subsampling
  - Language model

Your first task - to fill **`#<TODO>`** parts in AudioAdapter and AudioLLM classes.

##### Grading Criteria:
* **2 points:** Completed *<TODO>* code in the `AudioAdapter` and in the `AudioLLM`, `AudioEmbeddingInsertionTests` passed
* **1 point:** Completed *<TODO>* code in the `AudioAdapter`, `AudioEmbeddingInsertionTests` failed

In [ ]:
@dataclass
class ModelConfig:
    """Configuration for the Audio LLM model."""
    
    whisper_model: str = "openai/whisper-small"
    llm_model: str = "Qwen/Qwen3-0.6B"
    
    # Audio adapter
    adapter_hidden_dim: int = 1024 # corresponding to llm input dimension
    adapter_num_layers: int = 2
    adapter_dropout: float = 0.1
    subsample_factor: int = 4

    # Training strategy
    freeze_whisper: bool = True  # Freeze Whisper encoder
    freeze_llm: bool = True  # Set to True for adapter-only training (no LoRA needed)


In [ ]:
class AudioAdapter(nn.Module):
    def __init__(
        self,
        whisper_dim: int,
        llm_dim: int,
        hidden_dim: int = 2048,
        num_layers: int = 2,
        dropout: float = 0.1,
        subsample_factor: int = 1
    ):
        """
        Simple MLP Adapter with configurable subsample_factor
        """
        super().__init__()
        self.subsample_factor = subsample_factor

        # Set input_dim according to subsample_factor and whisper_dim
        input_dim = whisper_dim * subsample_factor
        self.input_proj = nn.Linear(input_dim, hidden_dim)

        self.layers = nn.ModuleList([
            nn.Sequential(
                nn.LayerNorm(hidden_dim),
                nn.Linear(hidden_dim, hidden_dim),
                nn.GELU(),
                nn.Dropout(dropout)
            )
            for _ in range(num_layers - 1)
        ])

        self.output_proj = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, llm_dim)
        )

    def forward(self, whisper_features: torch.Tensor) -> torch.Tensor:
        if self.subsample_factor > 1:
            batch_size, seq_len, dim = whisper_features.shape
            remainder = seq_len % self.subsample_factor
            if remainder != 0:
                whisper_features = whisper_features[:, :-remainder, :]
                seq_len = whisper_features.shape[1]
            new_seq_len = seq_len // self.subsample_factor
            x = whisper_features.reshape(batch_size, new_seq_len, dim * self.subsample_factor)
        else:
            x = whisper_features

        # Applying input projection
        x = self.input_proj(x)

        for layer in self.layers:
            x = x + layer(x)

        # Applying output projection
        x = self.output_proj(x)
        return x

In [ ]:
AUDIO_TOKEN = "<|audio_token|>"

class AudioLLM(nn.Module):
    """
    Audio Language Model combining Whisper encoder with a text LLM.
    
    Architecture:
    1. Whisper encoder processes audio
    2. Audio adapter projects features to LLM space
    3. LLM generates text conditioned on audio features
    """
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.model_config = config
        
        # Load Whisper encoder
        print(f"Loading Whisper model: {config.whisper_model}")
        whisper = WhisperModel.from_pretrained(config.whisper_model)
        self.whisper_encoder = whisper.encoder
        whisper_dim = self.whisper_encoder.config.d_model
        
        if config.freeze_whisper:
            for param in self.whisper_encoder.parameters():
                param.requires_grad = False
            print("Whisper encoder frozen")
        
        # Load LLM
        print(f"Loading LLM: {config.llm_model}")
        self.llm = AutoModelForCausalLM.from_pretrained(
            config.llm_model,
            torch_dtype=torch.bfloat16
        )

        # Adding special audio token
        self.tokenizer = AutoTokenizer.from_pretrained(config.llm_model)
        self.tokenizer.add_special_tokens({"additional_special_tokens": [AUDIO_TOKEN]})
        self.audio_token_id = self.tokenizer(AUDIO_TOKEN, add_special_tokens=False).input_ids[0]
        
        # Ensure tokenizer has pad token
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            
        llm_dim = self.llm.config.hidden_size
        
        # Audio adapter (initialize before freezing/LoRA)
        print("Initializing audio adapter")
        self.audio_adapter = AudioAdapter(
            whisper_dim=whisper_dim,
            llm_dim=llm_dim,
            hidden_dim=config.adapter_hidden_dim,
            num_layers=config.adapter_num_layers,
            dropout=config.adapter_dropout,
            subsample_factor=config.subsample_factor
        )
        
        # Freeze LLM if specified
        if config.freeze_llm:
            for param in self.llm.parameters():
                param.requires_grad = False
            print("✓ LLM fully frozen (only adapter will be trainable)")
        
        # Set config attribute to LLM's config (required by Trainer)
        self.config = self.llm.config
        
    def get_device(self):
        return next(self.llm.parameters()).device

    def get_dtype(self):
        return next(self.llm.parameters()).dtype
    
    def encode_audio(self, audio_values: torch.Tensor) -> torch.Tensor:
        """
        Encode audio through Whisper and adapter.
        For simplification, we are using fixed 30s-padding 
        as in original whisper and won't use audios longer 
        than 30 seconds in our training.
        
        Args:
            audio_values: [batch_size, audio_length]
        Returns:
            audio_embeds: [batch_size, num_audio_tokens, llm_dim]
        """
        # Ensure audio is on correct device and dtype
        whisper_dtype = self.get_dtype()
        whisper_device = self.get_device()
        audio_values = audio_values.to(device=whisper_device, dtype=whisper_dtype)
        
        # Whisper encoding
        with torch.no_grad() if self.model_config.freeze_whisper else torch.enable_grad():
            whisper_outputs = self.whisper_encoder(audio_values)
            whisper_features = whisper_outputs.last_hidden_state
        
        # Project to LLM space
        audio_embeds = self.audio_adapter(whisper_features)
        return audio_embeds
    
    @staticmethod
    def insert_audio_embeds(
        audio_embeds: torch.Tensor, 
        text_embeds: torch.Tensor, 
        input_ids: torch.Tensor,
        labels: Optional[torch.Tensor],
        audio_token_id: int
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """
        Insert audio embeddings into text embeddings at positions marked by audio_token_id.
        
        Args:
            audio_embeds: [batch_size, num_audio_tokens, llm_dim]
            text_embeds: [batch_size, text_seq_len, llm_dim]
            input_ids: [batch_size, text_seq_len] - contains audio_token_id at positions to replace
            labels: [batch_size, text_seq_len] - optional labels to adjust
            audio_token_id: int - ID of the audio token placeholder
        
        Returns:
            combined_embeds: [batch_size, new_seq_len, llm_dim] where audio_token_id positions are replaced
            combined_labels: [batch_size, new_seq_len] with -100 at audio positions (if labels provided)
        """
        batch_size, num_audio_tokens, llm_dim = audio_embeds.shape
        text_seq_len = text_embeds.shape[1]
        device = audio_embeds.device
        dtype = audio_embeds.dtype
        
        combined_embeds_list = []
        combined_labels_list = []
        
        for b in range(batch_size):
            # Find the position of audio_token_id in this batch item
            audio_token_positions = (input_ids[b] == audio_token_id).nonzero(as_tuple=True)[0]
            
            if len(audio_token_positions) == 0:
                # No audio token found, just use text embeddings as-is
                combined_embeds_list.append(text_embeds[b])
                if labels is not None:
                    combined_labels_list.append(labels[b])
                continue
            
            # Get the position of the (first) audio token placeholder
            audio_pos = audio_token_positions[0].item()
            
            # Build combined embeddings:
            # [text_before_audio] + [audio_embeds] + [text_after_audio]
            text_before = text_embeds[b, :audio_pos]  # [audio_pos, dim]
            text_after = text_embeds[b, audio_pos + 1:]  # [text_seq_len - audio_pos - 1, dim]
            audio = audio_embeds[b]  # [num_audio_tokens, dim]
            
            combined = torch.cat([text_before, audio, text_after], dim=0)
            combined_embeds_list.append(combined)
            
            # Build combined labels if provided
            if labels is not None:
                labels_before = labels[b, :audio_pos]  # [audio_pos]
                labels_after = labels[b, audio_pos + 1:]  # [text_seq_len - audio_pos - 1]
                # Audio positions should have -100 (ignored in loss)
                audio_labels = torch.full((num_audio_tokens,), -100, device=device, dtype=labels.dtype)
                
                combined_labels = torch.cat([labels_before, audio_labels, labels_after], dim=0)
                combined_labels_list.append(combined_labels)
        
        # Stack all batch items (they should all have the same length now)
        combined_embeds = torch.stack(combined_embeds_list, dim=0)
        
        if labels is not None:
            combined_labels = torch.stack(combined_labels_list, dim=0)
        else:
            combined_labels = None
        
        return combined_embeds, combined_labels
    
    def forward(
        self,
        audio_values: torch.Tensor,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None
    ) -> dict:
        """
        Forward pass for training.
        
        Args:
            audio_values: [batch_size, audio_length]
            input_ids: [batch_size, text_seq_len] - text tokens
            attention_mask: [batch_size, text_seq_len]
            labels: [batch_size, text_seq_len] - for computing loss
        """
        batch_size = audio_values.shape[0]
        
        # Encode audio
        audio_embeds = self.encode_audio(audio_values)  # [B, audio_tokens, dim]
        num_audio_tokens = audio_embeds.shape[1]
        
        # Get text embeddings
        text_embeds = self.llm.get_input_embeddings()(input_ids)  # [B, text_tokens, dim]
        
        # Insert audio embeddings and adjust labels
        combined_embeds, combined_labels = AudioLLM.insert_audio_embeds(
            audio_embeds, text_embeds, input_ids, labels, self.audio_token_id
        )
        
        # Create combined attention mask
        if attention_mask is not None:
            # Prepend ones for audio embeddings to existing attention mask
            audio_attention = torch.ones(
                batch_size, num_audio_tokens,
                device=audio_embeds.device,
                dtype=attention_mask.dtype
            )
            combined_attention = torch.cat([audio_attention, attention_mask], dim=1)
        else:
            combined_attention = None
        
        # Forward through LLM
        outputs = self.llm(
            inputs_embeds=combined_embeds,
            attention_mask=combined_attention,
            labels=combined_labels,
            return_dict=True
        )
        
        return outputs

#### Audio Embedding Insertion Method

In the whole AudioLLM class you'll need to implement only this method, all other methods are implemented already.

This method is marked as static to make automatic testing easier, don't unmark it.

##### Overview

This method is the core of multimodal fusion in AudioLLM. It takes:
- **Text embeddings** with special audio placeholder tokens
- **Audio embeddings** from the audio encoder
- **Input IDs** showing where placeholders are located

And produces a **combined embedding sequence** where audio placeholders are replaced by actual audio embeddings.

##### Algorithm Steps

1. **Locate placeholders**: Find positions in `input_ids` that contain `audio_token_id`
2. **Replace with audio**: At each placeholder position, insert ALL audio embeddings
3. **Keep text embeddings**: All other positions keep their original text embeddings
4. **Mask labels**: Set labels to `-100` at audio positions (ignored in loss calculation)
5. **Pad sequences**: Ensure all batch items have the same length

##### Visual Example

See the diagram below for how the insertion works:
![](embeddings_visualization.png)

In [ ]:
class AudioEmbeddingInsertionTests:
    """Test suite for insert_audio_embeds method."""
    
    def __init__(self, insert_audio_embeds_func):
        """
        Args:
            insert_audio_embeds_func: The static method to test
        """
        self.insert_func = insert_audio_embeds_func
        self.audio_token_id = 99999
        self.llm_dim = 128
        
    def test_single_audio_token_per_sample(self):
        """Test with exactly one audio placeholder per sample."""
        print("\n✓ Test 1: Single audio placeholder per sample")
        
        batch_size = 2
        text_seq_len = 5
        num_audio_tokens = 15  # Typical for Ultravox
        
        audio_embeds = torch.randn(batch_size, num_audio_tokens, self.llm_dim)
        text_embeds = torch.randn(batch_size, text_seq_len, self.llm_dim)
        
        # Each sample has exactly 1 audio placeholder at different positions
        input_ids = torch.tensor([
            [1, 2, self.audio_token_id, 3, 4],      # Audio at position 2
            [5, 6, 7, self.audio_token_id, 8]       # Audio at position 3
        ])
        
        labels = torch.tensor([
            [10, 20, 30, 40, 50],
            [60, 70, 80, 90, 100]
        ])
        
        combined_embeds, combined_labels = self.insert_func(
            audio_embeds, text_embeds, input_ids, labels, self.audio_token_id
        )
        
        # Expected: 5 - 1 + 15 = 19 tokens per sequence
        expected_len = text_seq_len - 1 + num_audio_tokens
        assert combined_embeds.shape == (batch_size, expected_len, self.llm_dim), \
            f"Expected shape {(batch_size, expected_len, self.llm_dim)}, got {combined_embeds.shape}"
        assert combined_labels.shape == (batch_size, expected_len), \
            f"Expected labels shape {(batch_size, expected_len)}, got {combined_labels.shape}"
        
        # Check audio positions have -100 labels
        # Batch 0: audio at positions 2-16 (15 tokens)
        for i in range(2, 2 + num_audio_tokens):
            assert combined_labels[0, i] == -100, \
                f"Audio position {i} should have -100 label, got {combined_labels[0, i]}"
        
        # Batch 1: audio at positions 3-17 (15 tokens)
        for i in range(3, 3 + num_audio_tokens):
            assert combined_labels[1, i] == -100, \
                f"Audio position {i} should have -100 label, got {combined_labels[1, i]}"
        
        # Check non-audio positions keep original labels
        assert combined_labels[0, 0] == 10, "First position should keep original label"
        assert combined_labels[0, 1] == 20, "Second position should keep original label"
        assert combined_labels[0, 17] == 40, "Position after audio should keep original label"
        
        print(f"  ✓ Output shape: {combined_embeds.shape}")
        print(f"  ✓ Labels shape: {combined_labels.shape}")
        print(f"  ✓ Audio positions (15 tokens) masked with -100")
        print(f"  ✓ Non-audio positions preserve original labels")
        
        return True
    
    def test_audio_at_beginning(self):
        """Test with audio placeholder at the start of sequence."""
        print("\n✓ Test 2: Audio placeholder at sequence beginning")
        
        batch_size = 2
        text_seq_len = 6
        num_audio_tokens = 10
        
        audio_embeds = torch.randn(batch_size, num_audio_tokens, self.llm_dim)
        text_embeds = torch.randn(batch_size, text_seq_len, self.llm_dim)
        
        # Audio at position 0
        input_ids = torch.tensor([
            [self.audio_token_id, 1, 2, 3, 4, 5],
            [self.audio_token_id, 6, 7, 8, 9, 10]
        ])
        
        labels = torch.tensor([
            [100, 10, 20, 30, 40, 50],
            [200, 60, 70, 80, 90, 100]
        ])
        
        combined_embeds, combined_labels = self.insert_func(
            audio_embeds, text_embeds, input_ids, labels, self.audio_token_id
        )
        
        expected_len = text_seq_len - 1 + num_audio_tokens
        assert combined_embeds.shape == (batch_size, expected_len, self.llm_dim), \
            f"Expected shape {(batch_size, expected_len, self.llm_dim)}, got {combined_embeds.shape}"
        
        # First 10 positions should be -100
        for i in range(num_audio_tokens):
            assert combined_labels[0, i] == -100, \
                f"Audio position {i} at beginning should have -100"
        
        # Check that text labels follow after audio
        assert combined_labels[0, num_audio_tokens] == 10, \
            "First text label should appear after audio"
        
        print(f"  ✓ Audio at beginning handled correctly")
        print(f"  ✓ Text labels follow after audio embeddings")
        
        return True
    
    def test_audio_at_end(self):
        """Test with audio placeholder at the end of sequence."""
        print("\n✓ Test 3: Audio placeholder at sequence end")
        
        batch_size = 2
        text_seq_len = 6
        num_audio_tokens = 10
        
        audio_embeds = torch.randn(batch_size, num_audio_tokens, self.llm_dim)
        text_embeds = torch.randn(batch_size, text_seq_len, self.llm_dim)
        
        # Audio at last position
        input_ids = torch.tensor([
            [1, 2, 3, 4, 5, self.audio_token_id],
            [6, 7, 8, 9, 10, self.audio_token_id]
        ])
        
        labels = torch.tensor([
            [10, 20, 30, 40, 50, 100],
            [60, 70, 80, 90, 100, 200]
        ])
        
        combined_embeds, combined_labels = self.insert_func(
            audio_embeds, text_embeds, input_ids, labels, self.audio_token_id
        )
        
        expected_len = text_seq_len - 1 + num_audio_tokens
        assert combined_embeds.shape == (batch_size, expected_len, self.llm_dim), \
            f"Expected shape {(batch_size, expected_len, self.llm_dim)}, got {combined_embeds.shape}"
        
        # Last 10 positions should be -100
        for i in range(expected_len - num_audio_tokens, expected_len):
            assert combined_labels[0, i] == -100, \
                f"Audio position {i} at end should have -100"
        
        # Check that text labels come before audio
        assert combined_labels[0, 0] == 10, "First text label preserved"
        assert combined_labels[0, 4] == 50, "Last text label before audio preserved"
        
        print(f"  ✓ Audio at end handled correctly")
        print(f"  ✓ Text labels preserved before audio")
        
        return True
    
    def test_embedding_values_preserved(self):
        """Test that text and audio embeddings are correctly preserved."""
        print("\n✓ Test 4: Embedding values preservation")
        
        batch_size = 1
        text_seq_len = 4
        num_audio_tokens = 3
        
        # Create embeddings with known values
        audio_embeds = torch.ones(batch_size, num_audio_tokens, self.llm_dim) * 100
        text_embeds = torch.ones(batch_size, text_seq_len, self.llm_dim) * 50
        
        # Audio at position 2
        input_ids = torch.tensor([[1, 2, self.audio_token_id, 3]])
        labels = torch.tensor([[10, 20, 30, 40]])
        
        combined_embeds, combined_labels = self.insert_func(
            audio_embeds, text_embeds, input_ids, labels, self.audio_token_id
        )
        
        # Check text embeddings before audio
        assert torch.allclose(combined_embeds[0, 0], text_embeds[0, 0]), \
            "Text embedding at position 0 should be preserved"
        assert torch.allclose(combined_embeds[0, 1], text_embeds[0, 1]), \
            "Text embedding at position 1 should be preserved"
        
        # Check audio embeddings
        for i in range(num_audio_tokens):
            assert torch.allclose(combined_embeds[0, 2 + i], audio_embeds[0, i]), \
                f"Audio embedding {i} should be preserved at position {2 + i}"
        
        # Check text embedding after audio
        assert torch.allclose(combined_embeds[0, 5], text_embeds[0, 3]), \
            "Text embedding after audio should be preserved"
        
        print(f"  ✓ Text embeddings preserved correctly")
        print(f"  ✓ Audio embeddings inserted correctly")
        print(f"  ✓ Embedding values match expected")
        
        return True
    
    def test_batch_consistency(self):
        """Test that batch processing is consistent."""
        print("\n✓ Test 5: Batch consistency")
        
        batch_size = 3
        text_seq_len = 7
        num_audio_tokens = 15
        
        audio_embeds = torch.randn(batch_size, num_audio_tokens, self.llm_dim)
        text_embeds = torch.randn(batch_size, text_seq_len, self.llm_dim)
        
        # Each sample has audio at position 3
        input_ids = torch.tensor([
            [1, 2, 3, self.audio_token_id, 4, 5, 6],
            [7, 8, 9, self.audio_token_id, 10, 11, 12],
            [13, 14, 15, self.audio_token_id, 16, 17, 18]
        ])
        
        labels = torch.tensor([
            [10, 20, 30, 40, 50, 60, 70],
            [80, 90, 100, 110, 120, 130, 140],
            [150, 160, 170, 180, 190, 200, 210]
        ])
        
        combined_embeds, combined_labels = self.insert_func(
            audio_embeds, text_embeds, input_ids, labels, self.audio_token_id
        )
        
        expected_len = text_seq_len - 1 + num_audio_tokens
        
        # All batch items should have same length
        assert combined_embeds.shape[0] == batch_size, \
            f"Batch size should be {batch_size}"
        assert combined_embeds.shape[1] == expected_len, \
            f"All sequences should have length {expected_len}"
        
        # Check each batch item has audio masked correctly
        for b in range(batch_size):
            audio_start = 3
            for i in range(audio_start, audio_start + num_audio_tokens):
                assert combined_labels[b, i] == -100, \
                    f"Batch {b}, position {i} should be -100"
        
        print(f"  ✓ All batch items have consistent length: {expected_len}")
        print(f"  ✓ Audio masking consistent across batch")
        
        return True
    
    def run_all_tests(self):
        """Run all tests and report results."""
        print("="*70)
        print("Running Audio Embedding Insertion Tests")
        print("="*70)
        
        tests = [
            ("Single Audio Token Per Sample", self.test_single_audio_token_per_sample),
            ("Audio at Beginning", self.test_audio_at_beginning),
            ("Audio at End", self.test_audio_at_end),
            ("Embedding Values Preservation", self.test_embedding_values_preserved),
            ("Batch Consistency", self.test_batch_consistency),
        ]
        
        passed = 0
        failed = 0
        
        for test_name, test_func in tests:
            try:
                result = test_func()
                if result:
                    passed += 1
            except AssertionError as e:
                print(f"  ✗ FAILED: {e}")
                failed += 1
            except Exception as e:
                print(f"  ✗ ERROR: {e}")
                failed += 1
        
        print("\n" + "="*70)
        print(f"Test Results: {passed}/{len(tests)} passed")
        if failed == 0:
            print("✅ All tests passed! Your implementation is correct.")
        else:
            print(f"❌ {failed} test(s) failed. Please review your implementation.")
        print("="*70)
        
        return failed == 0

In [ ]:
test_suite = AudioEmbeddingInsertionTests(AudioLLM.insert_audio_embeds)
test_suite.run_all_tests()

#### Dataset Implementation
* We've implemented the model class at this point, it's time to implement Audio-based QA dataset, samples for which we've generated earlier
* Your goal, again - fill **`<TODO>`**'s

##### Grading Criteria:
* **1 point:** Completed *<TODO>* code in the `AudioInstructDataset` + demonstration of dataset samples with **correct** labels masking



In [ ]:
def create_prompt(
        tokenizer: AutoTokenizer,
        instruction: str,
        response: str,
        system_message: str = BASE_SYSTEM_MESSAGE
) -> str:
    messages = [
        {"role": "system", "content": system_message}
    ]
    messages.append({"role": "user", "content": instruction})

    if response is not None:
        messages.append({"role": "assistant", "content": response})
        add_generation_prompt = False
    else:
        add_generation_prompt = True

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=add_generation_prompt
    )

    return prompt

class AudioInstructDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        json_path: str,
        tokenizer,
        feature_extractor: WhisperFeatureExtractor,
        max_length: int = 512,
        is_generation_set: bool = False,
        num_samples: Optional[int] = None,
    ):
        with open(json_path, 'r') as f:
            self.data = json.load(f)

        if num_samples is not None:
            self.data = self.data[:num_samples]

        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.max_length = max_length
        self.is_generation_set = is_generation_set

        print(f"Loaded {len(self.data)} samples from {json_path}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx) -> dict:
        sample = self.data[idx]

        # Load audio from file path and resample to 16kHz if needed
        audio_path = sample["audio_path"]
        waveform, sr = librosa.load(audio_path, sr=16000)
        waveform = torch.tensor(waveform).unsqueeze(0)

        # Process audio with Whisper feature extractor
        audio_array = waveform.squeeze(0).numpy()
        audio_inputs = self.feature_extractor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt"
        )
        audio_values = audio_inputs.input_features.squeeze(0)

        # Create chat-formatted prompt with instruction and response
        instruction = f"{AUDIO_TOKEN}Based on the given audio, answer the question: {sample['question']}"
        response = sample["answer"]

        text = create_prompt(
            self.tokenizer,
            instruction=instruction,
            response=response,
        )

        # Tokenize
        tokenized = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="longest",
            return_tensors="pt"
        )

        input_ids = tokenized.input_ids.squeeze(0)
        attention_mask = tokenized.attention_mask.squeeze(0)

        # Mask tokens corresponding to system and user replic with -100 value
        # Find the position where the assistant response starts
        # We need to mask everything before the assistant's response (system + user messages)
        # For Qwen models, we look for the assistant header or similar pattern
        labels = input_ids.clone()
        
        # Create prompt without response to find where assistant response starts
        prompt_without_response = create_prompt(
            self.tokenizer,
            instruction=instruction,
            response=None,  # No response to get just the prompt
        )
        prompt_tokens = self.tokenizer(
            prompt_without_response,
            return_tensors="pt",
            add_special_tokens=False
        ).input_ids.squeeze(0)
        
        # Mask all tokens up to and including the prompt (before assistant response)
        prompt_length = len(prompt_tokens)
        labels[:prompt_length] = -100

        return {
            "audio_values": audio_values,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

#### Train model to overfit on small dataset

We've implemented model & dataset, now we need to test it:
* Instantiate model
* Move it to device
* Create dataset with small number of samples (10, for example)
* Create trainer according to seminar notebook
* Adjust hyperparams to reach `loss < 0.2`

##### Grading Criteria:
* **+1 point:** Reaching `loss < 0.2` on your overfit experiment

In [ ]:
# Model instantiation
config = ModelConfig()
model = AudioLLM(config)

if torch.cuda.is_available():
    device = torch.device("cuda")
    dtype = torch.bfloat16

    # Move all components to GPU
    model.llm = model.llm.to(device=device)
    model.whisper_encoder = model.whisper_encoder.to(device=device, dtype=dtype)
    model.audio_adapter = model.audio_adapter.to(device=device, dtype=dtype)

    print(f"Model moved to device: {device}, dtype: {dtype}")
else:
    print("Using CPU (no CUDA available)")


tokenizer = model.tokenizer
feature_extractor = WhisperFeatureExtractor.from_pretrained(config.whisper_model)

In [ ]:
dataset = AudioInstructDataset(
    json_path = 'instruct_data.json', # Path to your QA-saved Data with specified format,
    tokenizer = tokenizer,
    feature_extractor = feature_extractor,
    num_samples = 10,  # Use only 10 samples for overfitting experiment
)


In [ ]:
## Use Trainer to overfit on small sample
from transformers import Trainer, TrainingArguments

In [ ]:
## Use Trainer to overfit on small sample

# Freeze all parameters except audio adapter
for param in model.whisper_encoder.parameters():
    param.requires_grad = False
for param in model.llm.parameters():
    param.requires_grad = False
for param in model.audio_adapter.parameters():
    param.requires_grad = True

print(f'Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')
print(f'Total parameters: {sum(p.numel() for p in model.parameters())}')

# Training arguments for overfitting
training_args = TrainingArguments(
    output_dir='./overfit_experiment',
    max_steps=200,
    per_device_train_batch_size=1,
    learning_rate=1e-3,
    warmup_steps=20,
    logging_steps=10,
    save_steps=100000,
    eval_strategy='no',
    bf16=torch.cuda.is_available(),
    dataloader_num_workers=0,
    remove_unused_columns=False,
    report_to=['tensorboard'],
    logging_dir='./overfit_experiment/logs',
)

class AudioLLMTrainer(Trainer):
    """Custom Trainer for AudioLLM that handles device placement correctly."""
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        outputs = model(**inputs)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

trainer = AudioLLMTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Train the model
trainer.train()

In [ ]:
## Plot training loss
import matplotlib.pyplot as plt

# Get training history from trainer
training_history = trainer.state.log_history

# Extract loss values and steps
steps = [entry['step'] for entry in training_history if 'loss' in entry]
losses = [entry['loss'] for entry in training_history if 'loss' in entry]

plt.figure(figsize=(10, 6))
plt.plot(steps, losses, marker='o', markersize=3)
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.grid(True)
plt.show()

print(f'Final loss: {losses[-1] if losses else "N/A"}')
print(f'Min loss: {min(losses) if losses else "N/A"}')

---

### 3. Pretrained AudioLLM Evaluation (4 points)

In this section you'll need to: 
* Load pretrained checkpoint with architecture you've implemented

* Write generation method for the model

* Measure the quality of the model on MMLU dataset with *answer probability* - based accuracy


#### Grading Criteria:
* **+2 points:** Completing `generate` function and meaningfull answers on example audios
* **+2 point:** Completed *answer probability* - based accuracy calculation on mmlu_speech subset and reaching `accuracy > 0.4`

---

#### Generation function
First, you'll need to:
* Implement `get_audio_values` and `generate` functions
* Pass example audios into them with or without prompts and demonstrate meaningfull answers

In [ ]:
# Downloading audios we will use in examples

! gdown https://drive.google.com/uc?id=18-9If-0WZH0cPZ9MpQEwCrABA3BrTKBh
! gdown https://drive.google.com/uc?id=1Yv6B3BEMjjmkcK4ogcutyf6gR1QOl3kl

In [ ]:
import IPython
IPython.display.Audio('gagarin_eng.mp3')
# When did Gagarin fly into space?

In [ ]:
import IPython
IPython.display.Audio('gagarin.mp3')
# В каком году Гагарин полетел в космос?

In [ ]:
# Model instantiation
config = ModelConfig(
    whisper_model='openai/whisper-medium',
    llm_model="Qwen/Qwen2.5-1.5B-Instruct",
    adapter_hidden_dim=1536,
    adapter_num_layers=2,
    adapter_dropout=0.1,
    subsample_factor=4,
)
model = AudioLLM(config)

In [ ]:
from huggingface_hub import hf_hub_download

ckpt_path = hf_hub_download(
    repo_id="malex26/dls-course-model",
    filename="model.ckpt",
)

In [ ]:
ckpt = torch.load(ckpt_path, map_location='cpu')
model.load_state_dict(ckpt)
model.to(DEVICE)

In [ ]:
def get_audio_values(audio_path: str | Path, features_extractor: WhisperFeatureExtractor) -> torch.Tensor:
    """
    Extracting audio_values tensor from given audio_path
    """
    # Load audio using librosa and resample to 16kHz
    audio, sr = librosa.load(audio_path, sr=16000)
    
    # Process audio with Whisper feature extractor
    audio_inputs = features_extractor(
        audio,
        sampling_rate=16000,
        return_tensors="pt"
    )
    audio_values = audio_inputs.input_features  # [1, 128, time_frames]
    
    return audio_values

def generate(
    model: AudioLLM, 
    audio_values: torch.Tensor,
    user_prompt: str = "",
    system_prompt: str = "",
    max_new_tokens: int=128,
    **generation_kwargs,
) -> str:
    """
    Standard generation logic, but with addition of Audio Embeddings
    """
    model = model.eval()
    device = model.get_device()
    
    if not AUDIO_TOKEN in user_prompt:
        # model was trained with AUDIO_TOKEN before user text
        user_prompt = AUDIO_TOKEN + user_prompt

    # Create the prompt using chat template
    text = create_prompt(
        model.tokenizer,
        instruction=user_prompt,
        response=None,  # No response for generation
        system_message=system_prompt if system_prompt else BASE_SYSTEM_MESSAGE,
    )
    
    # Tokenize
    tokenized = model.tokenizer(text, return_tensors="pt")
    input_ids = tokenized.input_ids.to(device)

    with torch.no_grad(), torch.autocast(dtype=torch.bfloat16, device_type='cuda'):
        # Obtain audio embeds
        audio_values = audio_values.to(device)
        audio_embeds = model.encode_audio(audio_values)  # [1, num_audio_tokens, llm_dim]

        # Obtain text embeds
        text_embeds = model.llm.get_input_embeddings()(input_ids)  # [1, text_seq_len, llm_dim]

        # Combine them
        combined_embeds, _ = model.insert_audio_embeds(
            audio_embeds, text_embeds, input_ids, None, model.audio_token_id
        )

        # Create attention mask
        attention_mask = torch.ones(
            1, combined_embeds.shape[1],
            device=audio_embeds.device,
            dtype=torch.long
        )

        # Generate
        outputs = model.llm.generate(
            inputs_embeds=combined_embeds,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            pad_token_id=model.tokenizer.pad_token_id,
            eos_token_id=model.tokenizer.eos_token_id,
            **generation_kwargs
        )

        # Decode tokens
        generated_text = model.tokenizer.decode(
            outputs[0],
            skip_special_tokens=True
        ).strip()


    return generated_text

    

In [ ]:
audio_values = get_audio_values('gagarin_eng.mp3', feature_extractor)

In [ ]:
res = generate(
    model=model,
    audio_values=audio_values,
    user_prompt=f'{AUDIO_TOKEN}Answer the question on audio.'
)

In [ ]:
print(res)

#### Grading Criteria:
* **2 points:** Completed `generate` & `get_audio_values` function and demonstrated meaningfull answers on example audios

---

#### Measure the quality of the model on MMLU dataset with *answer probability* - based accuracy
In this punct you'll evaluate the pretrained AudioLLM model on MMLU-Speech using **probability-based answer extraction**, similar to the method in original [MMLU paper](https://arxiv.org/pdf/2009.03300).

### Method

Instead of generating and parsing text, directly extract the answer by:

1. Create prompt ending with `"Answer:"`
2. Get model's logits for the **next token**
3. Extract probabilities for tokens `" A"`, `" B"`, `" C"`, `" D"` (note: with leading space)
4. Select the option with **highest probability**

**Your goal is**:
* implement this evaluation method
* measure quality on mmlu-speech dataset and reach `accuracy > 0.4` on the `high_school_psychology` subset

#### Grading Criteria:
* **2 points:** Creating probability-based validation function and obtained `accuracy > 0.4` on the `high_school_psychology` subset
* **1 points:** Creating any validation function and obtained `accuracy > 0.3` on the `high_school_psychology` subset

In [ ]:
! pip install datasets==2.21.0 -q

In [ ]:
from datasets import load_dataset

# Dataset downloading, takes about 5 minutes in kaggle notebooks
ds = load_dataset("mistralai/mmlu_speech", split='test')


In [ ]:
# Filter to high_school_psychology subset
ds_psychology = ds.filter(lambda x: x['subject'] == 'high_school_psychology')
print(f'Number of high_school_psychology samples: {len(ds_psychology)}')

# Define answer tokens (with leading space as per MMLU paper)
ANSWER_TOKENS = [' A', ' B', ' C', ' D']
answer_token_ids = [model.tokenizer.encode(tok, add_special_tokens=False)[0] for tok in ANSWER_TOKENS]
print(f'Answer token IDs: {dict(zip(ANSWER_TOKENS, answer_token_ids))}')

def evaluate_mmlu_sample_probability(
    model: AudioLLM,
    sample: dict,
    feature_extractor: WhisperFeatureExtractor,
) -> tuple[int, int]:
    """
    Evaluate a single MMLU sample using probability-based answer extraction.
    
    Returns:
        (predicted_answer_idx, correct_answer_idx)
    """
    model.eval()
    device = model.get_device()
    
    # Get audio from sample
    audio_array = sample['audio']['array']
    audio_sr = sample['audio']['sampling_rate']
    
    # Resample if needed
    if audio_sr != 16000:
        audio_array = librosa.resample(audio_array, orig_sr=audio_sr, target_sr=16000)
    
    # Process audio with feature extractor
    audio_inputs = feature_extractor(
        audio_array,
        sampling_rate=16000,
        return_tensors="pt"
    )
    audio_values = audio_inputs.input_features.to(device)
    
    # Create prompt ending with "Answer:"
    user_prompt = f"{AUDIO_TOKEN}Listen to the question and answer choices. Answer:"
    
    text = create_prompt(
        model.tokenizer,
        instruction=user_prompt,
        response=None,
        system_message=BASE_SYSTEM_MESSAGE,
    )
    
    tokenized = model.tokenizer(text, return_tensors="pt")
    input_ids = tokenized.input_ids.to(device)
    
    with torch.no_grad(), torch.autocast(dtype=torch.bfloat16, device_type='cuda'):
        # Encode audio
        audio_embeds = model.encode_audio(audio_values)
        
        # Get text embeddings
        text_embeds = model.llm.get_input_embeddings()(input_ids)
        
        # Combine embeddings
        combined_embeds, _ = model.insert_audio_embeds(
            audio_embeds, text_embeds, input_ids, None, model.audio_token_id
        )
        
        # Create attention mask
        attention_mask = torch.ones(
            1, combined_embeds.shape[1],
            device=device,
            dtype=torch.long
        )
        
        # Forward pass to get logits
        outputs = model.llm(
            inputs_embeds=combined_embeds,
            attention_mask=attention_mask,
            return_dict=True
        )
        
        # Get logits for the last position (next token prediction)
        last_logits = outputs.logits[0, -1, :]  # [vocab_size]
        
        # Extract probabilities for answer tokens
        answer_logits = last_logits[answer_token_ids]
        answer_probs = torch.softmax(answer_logits, dim=0)
        
        # Get predicted answer
        predicted_idx = answer_probs.argmax().item()
        
    # Get correct answer index
    correct_idx = sample['answer']
    
    return predicted_idx, correct_idx

# Evaluate on high_school_psychology subset
correct = 0
total = 0

for i, sample in enumerate(tqdm(ds_psychology, desc='Evaluating MMLU')):
    try:
        pred_idx, correct_idx = evaluate_mmlu_sample_probability(
            model, sample, feature_extractor
        )
        if pred_idx == correct_idx:
            correct += 1
        total += 1
    except Exception as e:
        print(f'Error on sample {i}: {e}')
        continue

accuracy = correct / total if total > 0 else 0
print(f'\nResults on high_school_psychology subset:')
print(f'Correct: {correct}/{total}')
print(f'Accuracy: {accuracy:.4f}')